In [7]:
import tensorflow as tf
from tensorflow import keras

In [11]:
# 入力: 2次元行列1つ
# 計算, 出力: input * w(trainable) + b(trainable), ただし、別変数をおいて出力
class MyLayer2(tf.keras.layers.Layer):
    def __init__(self, **kwargs):                   # ここではinputの形状による初期化を行わない
        super(MyLayer2, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.shape = input_shape
        self.w = self.add_weight(
            "weight", shape=input_shape, dtype='float32', initializer=tf.constant_initializer(1.0), trainable=True
        )
        self.b = self.add_weight(
            "bias", shape=input_shape, dtype='float32', initializer=tf.constant_initializer(2.0), trainable=True
        )
        
    def call(self, inputs):
        output = inputs * self.w + self.b
        return output

In [30]:
# 入力: 2x2, 2x2のふたつの行列f, s
# 出力: 2x2x2行列 [f*2, s^2]
class MyLayer3(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MyLayer3, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.shape = input_shape
        print(self.shape)
        
    def call(self, inputs):
        f, s = inputs
        output = tf.zeros(shape=[2,
                                 f.shape[0],
                                 f.shape[1]])
        output[0,:,:] = f * 2
        output[1,:,:] = s * s

In [8]:
class MyLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.shape = input_shape
        w_init = tf.constant_initializer(2.0)
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_shape[0] + 1), dtype="float32"),
            trainable=True,
        )
        b_init = tf.constant_initializer(10.0)
        self.b = tf.Variable(
            initial_value=b_init(shape=(input_shape[0] + 1,), dtype="float32"), trainable=True
        )
        self.built = True

    def call(self, inputs):
        return inputs * self.w[0:self.shape[0]] + self.b[0:self.shape[0]]

In [9]:
x = tf.ones([1])
my_layer = MyLayer()
y = my_layer(x)
print(y)

tf.Tensor([12.], shape=(1,), dtype=float32)


In [19]:
x = tf.ones([2, 2])
print(x)
my_layer_2 = MyLayer2()
y = my_layer_2(x)
print(y)

TypeError: Dimension value must be integer or None or have an __index__ method, got value 'TensorShape([3, 3])' with type '<class 'tensorflow.python.framework.tensor_shape.TensorShape'>'

In [31]:
x1 = tf.ones([3, 3])
x2 = tf.fill([3, 3], 3.0)
print(x1)
print(x2)
mylayer_3 = MyLayer3()
y = mylayer_3([x1, x2])
print(y)

tf.Tensor(
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[3. 3. 3.]
 [3. 3. 3.]
 [3. 3. 3.]], shape=(3, 3), dtype=float32)
ListWrapper([TensorShape([3, 3]), TensorShape([3, 3])])


TypeError: Exception encountered when calling layer 'my_layer3_4' (type MyLayer3).

'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

Call arguments received by layer 'my_layer3_4' (type MyLayer3):
  • inputs=['tf.Tensor(shape=(3, 3), dtype=float32)', 'tf.Tensor(shape=(3, 3), dtype=float32)']

In [13]:
my_layer_2.weights

[<tf.Variable 'my_layer2_3/weight:0' shape=(2, 2) dtype=float32, numpy=
 array([[1., 1.],
        [1., 1.]], dtype=float32)>,
 <tf.Variable 'my_layer2_3/bias:0' shape=(2, 2) dtype=float32, numpy=
 array([[2., 2.],
        [2., 2.]], dtype=float32)>]

In [14]:
from keras.models import Sequential

model = Sequential()
model.add(MyLayer2())
model.build(input_shape=[2, 2])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-1),
              loss=tf.keras.losses.MSE,)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_layer2_4 (MyLayer2)      (2, 2)                    8         
                                                                 
Total params: 8
Trainable params: 8
Non-trainable params: 0
_________________________________________________________________


In [16]:
x = [
    [[1, 2], [3, 4]], [[2, 4], [6, 8]]
]
y = [
    [[3, 5], [4, 6]], [[6, 10], [8, 12]]
]

model.fit(x, y, epochs=200)
model.evaluate(x, y)

Epoch 1/200
1/1 [==============================] - 0s 14ms/step - loss: 2.0634
Epoch 2/200
1/1 [==============================] - 0s 11ms/step - loss: 1.0529
Epoch 3/200
1/1 [==============================] - 0s 12ms/step - loss: 0.7753
Epoch 4/200
1/1 [==============================] - 0s 11ms/step - loss: 0.9485
Epoch 5/200
1/1 [==============================] - 0s 12ms/step - loss: 1.0138
Epoch 6/200
1/1 [==============================] - 0s 10ms/step - loss: 0.9749
Epoch 7/200
1/1 [==============================] - 0s 10ms/step - loss: 0.9531
Epoch 8/200
1/1 [==============================] - 0s 12ms/step - loss: 0.8868
Epoch 9/200
1/1 [==============================] - 0s 11ms/step - loss: 0.7109
Epoch 10/200
1/1 [==============================] - 0s 10ms/step - loss: 0.5317
Epoch 11/200
1/1 [==============================] - 0s 12ms/step - loss: 0.4807
Epoch 12/200
1/1 [==============================] - 0s 10ms/step - loss: 0.5455
Epoch 13/200
1/1 [==============================]

1.4375132195709739e-09

In [17]:
model.get_weights()

[array([[2.9999778, 2.4999502],
        [1.3333434, 1.5000091]], dtype=float32),
 array([[ 5.7377194e-05,  1.2237382e-04],
        [-7.3388735e-05, -7.5164397e-05]], dtype=float32)]